# Summary

Do I sleep less on nights when I play Ultimate frisbee?

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nba_py

sns.set_context('poster')
import plotly.offline as py
import plotly.graph_objs as go

In [2]:
py.init_notebook_mode(connected=True)

In [3]:
data_path = os.path.join(os.getcwd(), os.pardir, 'data', 'interim', 'sleep_data.csv')
df_sleep = pd.read_csv(data_path, index_col='shifted_datetime', parse_dates=True)

In [4]:
df_sleep.index += pd.Timedelta(hours=12)

In [5]:
sleep_day = df_sleep.resample('1D').sum().fillna(0)

In [6]:
data_path = os.path.join(os.getcwd(), os.pardir, 'data', 'interim', 'activity_data.csv')
df_activity = pd.read_csv(data_path, index_col='datetime', parse_dates=True)

In [7]:
df_activity.columns

Index(['calories', 'caloriesBMR', 'steps', 'distance', 'minutesSedentary',
       'minutesLightlyActive', 'minutesFairlyActive', 'minutesVeryActive'],
      dtype='object')

In [8]:
toplot = df_activity['minutesVeryActive']

data = []

data.append(
    go.Scatter(
        x=toplot.index,
        y=toplot.values,
        name='Minutes Very Active'
    )
)


layout = go.Layout(
    title="Daily Very Active Minutes",
    yaxis=dict(
        title='Minutes'
    ),
)

fig = {
    'data': data,
    'layout': layout,
}

py.iplot(fig, filename='DailyVeryActiveMinutes')

Lots of variation here. Can I relate this to Ultimate frisbee? Spring hat league started on April 14 and played once a week on Fridays until May 19. Meanwhile, I started playing summer club league on May 2. We play twice a week on Tuesdays and Thursdays with our last game on August 17.

In [9]:
dayofweek = df_activity.index.dayofweek

In [11]:
index_summerleague = df_activity.index >= '2017-05-02'

In [12]:
df_activity_summer = df_activity[index_summerleague]

In [13]:
summer_dayofweek = df_activity_summer.index.dayofweek

In [15]:
df_activity_summer['dayofweek'] = summer_dayofweek

In [16]:
df_activity_summer.groupby('dayofweek').mean()

,calories,caloriesBMR,steps,distance,minutesSedentary,minutesLightlyActive,minutesFairlyActive,minutesVeryActive
dayofweek,,,,,,,,
0,2900.615385,1727.000000,13510.461538,6.470530,640.153846,284.076923,39.615385,32.307692
1,2751.500000,1626.928571,13432.571429,6.440832,647.428571,247.000000,41.714286,32.428571
2,2745.230769,1727.000000,11590.923077,5.537409,665.769231,257.538462,37.076923,23.076923
3,2994.538462,1727.000000,15256.769231,7.295554,696.076923,275.923077,53.615385,35.615385
4,2970.692308,1727.000000,14798.923077,7.103748,644.923077,285.846154,41.538462,33.307692
5,3052.000000,1727.000000,14749.384615,7.045185,516.846154,378.230769,41.923077,16.538462
6,2957.538462,1727.000000,13567.307692,6.485205,578.538462,344.615385,36.076923,22.846154


Monday = 0, Sunday = 6. Saturday, Sunday, and Wednesday stand out as days where I have fewer Very Active minutes, but there is no obvious evidence that Tuesday and Thursday are days where I am running around chasing plastic for 1-2 hours. I suspect that part of the challenge here is that I ride my bike to work every day. It's 15 - 20 minutes each way, so if that time on the bike goes in to the "Very Active" bin according to Fitbit, then it will be mixed in with ultimate frisbee minutes. I might be able to filter out bike rides by looking at the start time of each activity. However, I will need to go back to the Fitbit API to extract that information.

In [17]:
df_activity_summer.groupby('dayofweek').std()

,calories,caloriesBMR,steps,distance,minutesSedentary,minutesLightlyActive,minutesFairlyActive,minutesVeryActive
dayofweek,,,,,,,,
0,235.444805,0.000,3252.885888,1.558011,103.987055,60.164305,23.464649,17.726555
1,710.455298,374.433,4464.648538,2.153538,188.691737,98.802055,20.700905,19.393553
2,148.270110,0.000,1660.270784,0.789373,104.050592,46.421287,13.406849,8.911243
3,312.696769,0.000,4191.693257,2.001949,95.629896,49.026288,27.717439,16.795527
4,237.309357,0.000,3365.512800,1.642800,102.469883,47.303710,18.724384,18.652009
5,345.327381,0.000,4394.001395,2.100775,98.811138,84.196352,37.785494,12.169740
6,240.380468,0.000,3268.354157,1.565045,106.219596,53.914343,18.679140,19.398652
